In [1]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=32489 sha256=9a39bcc742b2e48cdf8e2f530887e548f7b2d76f29bc01fb4a81c0272e6c4cf2
  Stored in directory: /tmp/pip-ephem-wheel-cache-t35342ah/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [17]:
from spuco.invariant_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_MEDIUM

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [18]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [19]:
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

In [20]:
from torch.optim import SGD

erm = ERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    batch_size=32,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4),
    device=device,
    verbose=True
)

In [21]:
erm.train()

Epoch 4: 100%|██████████| 1875/1875 [00:06<00:00, 299.67batch/s, accuracy=81.25%, loss=0.599] 


In [22]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 93.14420803782505
Group (0, 1) Test Accuracy: 91.48936170212765
Group (0, 2) Test Accuracy: 90.30732860520095
Group (0, 3) Test Accuracy: 88.17966903073287
Group (0, 4) Test Accuracy: 84.39716312056737
Group (1, 0) Test Accuracy: 87.28606356968216
Group (1, 1) Test Accuracy: 87.77506112469437
Group (1, 2) Test Accuracy: 86.76470588235294
Group (1, 3) Test Accuracy: 87.00980392156863
Group (1, 4) Test Accuracy: 87.99019607843137
Group (2, 0) Test Accuracy: 76.8
Group (2, 1) Test Accuracy: 70.13333333333334
Group (2, 2) Test Accuracy: 69.86666666666666
Group (2, 3) Test Accuracy: 70.4
Group (2, 4) Test Accuracy: 73.52941176470588
Group (3, 0) Test Accuracy: 80.90452261306532
Group (3, 1) Test Accuracy: 85.8942065491184
Group (3, 2) Test Accuracy: 82.87153652392946
Group (3, 3) Test Accuracy: 83.87909319899245
Group (3, 4) Test Accuracy: 87.40554156171285
Group (4, 0) Test Accuracy: 71.53652392947103
Group (4, 1) Test Accuracy: 75.31486146095718
Group (4, 2) Te

{(0, 0): 93.14420803782505,
 (0, 1): 91.48936170212765,
 (0, 2): 90.30732860520095,
 (0, 3): 88.17966903073287,
 (0, 4): 84.39716312056737,
 (1, 0): 87.28606356968216,
 (1, 1): 87.77506112469437,
 (1, 2): 86.76470588235294,
 (1, 3): 87.00980392156863,
 (1, 4): 87.99019607843137,
 (2, 0): 76.8,
 (2, 1): 70.13333333333334,
 (2, 2): 69.86666666666666,
 (2, 3): 70.4,
 (2, 4): 73.52941176470588,
 (3, 0): 80.90452261306532,
 (3, 1): 85.8942065491184,
 (3, 2): 82.87153652392946,
 (3, 3): 83.87909319899245,
 (3, 4): 87.40554156171285,
 (4, 0): 71.53652392947103,
 (4, 1): 75.31486146095718,
 (4, 2): 71.03274559193954,
 (4, 3): 76.76767676767676,
 (4, 4): 79.79797979797979}

In [23]:
evaluator.worst_group_accuracy

((2, 2), 69.86666666666666)

In [24]:
evaluator.average_accuracy

83.1767673778784

In [25]:
evaluator.evaluate_spurious_task()

20.48

In [26]:
import random 
from torch.utils.data import Subset

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

group_balanced_set = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.2,
    classes=classes,
    train=True
)
group_balanced_set.initialize()
group_balanced_set = Subset(group_balanced_set, [i for key in group_balanced_set.group_partition.keys() for i in group_balanced_set.group_partition[key][:240]])

In [27]:
from spuco.finetune import DFR 
from torch.optim import SGD

dfr = DFR(
    group_balanced_dataset=group_balanced_set,
    model=model,
    num_epochs=10000,
    lr=1,
    weight_decay=1e-3,
    verbose=True,
    device=device
)

In [28]:
dfr.train()

DFR: 100%|██████████| 10000/10000 [00:04<00:00, 2121.79it/s, acc=90.8, epoch=9000]


In [29]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 97.63593380614657
Group (0, 1) Test Accuracy: 97.87234042553192
Group (0, 2) Test Accuracy: 97.39952718676123
Group (0, 3) Test Accuracy: 97.87234042553192
Group (0, 4) Test Accuracy: 96.21749408983452
Group (1, 0) Test Accuracy: 90.70904645476773
Group (1, 1) Test Accuracy: 90.70904645476773
Group (1, 2) Test Accuracy: 92.15686274509804
Group (1, 3) Test Accuracy: 91.91176470588235
Group (1, 4) Test Accuracy: 89.46078431372548
Group (2, 0) Test Accuracy: 87.2
Group (2, 1) Test Accuracy: 82.13333333333334
Group (2, 2) Test Accuracy: 84.8
Group (2, 3) Test Accuracy: 82.4
Group (2, 4) Test Accuracy: 86.36363636363636
Group (3, 0) Test Accuracy: 89.19597989949749
Group (3, 1) Test Accuracy: 90.93198992443325
Group (3, 2) Test Accuracy: 88.91687657430731
Group (3, 3) Test Accuracy: 87.40554156171285
Group (3, 4) Test Accuracy: 90.17632241813602
Group (4, 0) Test Accuracy: 86.90176322418137
Group (4, 1) Test Accuracy: 86.64987405541562
Group (4, 2) Test Accuracy:

{(0, 0): 97.63593380614657,
 (0, 1): 97.87234042553192,
 (0, 2): 97.39952718676123,
 (0, 3): 97.87234042553192,
 (0, 4): 96.21749408983452,
 (1, 0): 90.70904645476773,
 (1, 1): 90.70904645476773,
 (1, 2): 92.15686274509804,
 (1, 3): 91.91176470588235,
 (1, 4): 89.46078431372548,
 (2, 0): 87.2,
 (2, 1): 82.13333333333334,
 (2, 2): 84.8,
 (2, 3): 82.4,
 (2, 4): 86.36363636363636,
 (3, 0): 89.19597989949749,
 (3, 1): 90.93198992443325,
 (3, 2): 88.91687657430731,
 (3, 3): 87.40554156171285,
 (3, 4): 90.17632241813602,
 (4, 0): 86.90176322418137,
 (4, 1): 86.64987405541562,
 (4, 2): 85.64231738035265,
 (4, 3): 88.38383838383838,
 (4, 4): 91.66666666666667}

In [30]:
evaluator.worst_group_accuracy

((2, 1), 82.13333333333334)